## POS tagging using modified Viterbi

This file contains 3 modifications to the Vanilla Viterbi algorithm as follows:
* **Transition Based Technique** : This sets the tag of the word based on the maximum of the Transition probabilities.
* **Transition and Reverse Transition Based Technique** : This sets the tag of the word based on the maximum of Transition and Reverse Transition probabilities. Reverse Transition probability is probability of t2 given t1 **follows** t2.
* **Rule Based Technique** : This sets the tag of the word based on certain rules

Some points considered:
* Random seeds have been used so that the same test-train split is achieved every time this file is executed
* The tag list is sorted so that everytime, the tag for the unknown words is the same
* A set of 5 random sentences, again with a random seed, has been considered for evaluating the tag accuracy
* The Test_sentences file has been read and 3 relevant sentences have been considered for showing the modification corrections. So if this file is not in the same folder, there will be errors
* Manual tagging of the sentences in the Test_sentences has been done and accuracy has been calculated for the same 3 relevant sentences
* Conclusions regarding accuracies and modification corrections have been given at the end in a tabular form.

### Data Preparation

In [39]:
#Importing libraries
import nltk
import random
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pprint, time
import re
import gc

In [40]:
gc.collect()

0

In [41]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [42]:
nltk_data[:20]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

### Split data into train and test sets

In [43]:
#Set random seed to make sure the train and test set are the same each time this call is executed for predictable results
train_set, test_set = train_test_split(nltk_data, test_size=0.05, random_state=200)

print("Total tagged sentences in data set = {0}".format(len(nltk_data)))
print("Tagged sentences in training set = {0}".format(len(train_set)))
print("Tagged sentences in test set = {0}".format(len(test_set)))

Total tagged sentences in data set = 3914
Tagged sentences in training set = 3718
Tagged sentences in test set = 196


In [44]:
#Get the tagged tuples in the train and the test set
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup for sent in test_set for tup in sent]

### Build the Vanilla Viterbi based POS tagger

#### Explore train data to see how many unique words and tags are there

In [45]:
# Get the unique tokens/words and unique POS tags in the train set 
tokens = set([pair[0] for pair in train_tagged_words])

# Sort the tag set so that later on, the Viterbi algorithm always picks up the same first tag for unknown words. 
# Else the accuracy keeps changing
tags = sorted(set([pair[1] for pair in train_tagged_words]))

no_of_unique_tokens = len(tokens)
no_of_unique_tags = len(tags)

#print("Unique words in data set = {0}".format(tokens))
print("Number of unique words in train data set = {0}".format(no_of_unique_tokens))
print("Unique tags in train data set = {0}".format(tags))
print("Number of unique tags in train data set = {0}".format(no_of_unique_tags))

Number of unique words in train data set = 12076
Unique tags in train data set = ['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']
Number of unique tags in train data set = 12


#### Compute Emission and Transition Probabilities

In [46]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)    
    return (count_w_given_tag, count_tag)

In [47]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1Precedes(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [48]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)
tags_matrix = np.zeros((no_of_unique_tags, no_of_unique_tags), dtype='float32')
for i, t1 in enumerate(tags):
    for j, t2 in enumerate(tags):
        t2_given_t1Results = t2_given_t1Precedes(t2, t1)
        tags_matrix[i, j] = t2_given_t1Results[0]/t2_given_t1Results[1]

In [49]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = tags, index=tags)
tags_df

,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
.,0.092866,0.044406,0.090344,0.052243,0.058458,0.174293,0.222032,0.080886,0.065574,0.002432,0.089173,0.027202
ADJ,0.064929,0.067388,0.078210,0.004427,0.016560,0.005083,0.698639,0.019839,0.000656,0.010985,0.012461,0.020823
ADP,0.039804,0.106499,0.016967,0.013232,0.000854,0.323872,0.323445,0.063067,0.068082,0.001494,0.008324,0.034361
ADV,0.135610,0.130305,0.118700,0.078581,0.006631,0.068966,0.030172,0.032825,0.015915,0.014257,0.345822,0.022215
CONJ,0.034451,0.119181,0.053073,0.055866,0.000466,0.119181,0.350093,0.041899,0.056797,0.005121,0.155028,0.008845
DET,0.017125,0.204896,0.009407,0.013025,0.000482,0.005548,0.637603,0.022552,0.003256,0.000241,0.039797,0.046069
NOUN,0.239953,0.012096,0.176595,0.016905,0.042445,0.013408,0.264145,0.009509,0.004700,0.044304,0.147229,0.028710
NUM,0.115852,0.033185,0.035852,0.002963,0.013926,0.003556,0.353778,0.183704,0.001481,0.027556,0.017778,0.210370
PRON,0.039330,0.074377,0.023364,0.034657,0.005452,0.009735,0.204829,0.007788,0.008178,0.012461,0.486371,0.093458
PRT,0.041585,0.087099,0.020629,0.010151,0.002292,0.102161,0.250164,0.057629,0.018337,0.001965,0.395219,0.012770


#### Vanilla Viterbi Algorithm

In [50]:
# Viterbi Heuristic
def Viterbi_Vanilla(words, T = tags):
    state = []
    unknown_word_tags = []
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            word_given_tagResults = word_given_tag(words[key], tag)
            emission_p = word_given_tagResults[0]/word_given_tagResults[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)        
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        if pmax == 0:
            unknown_word_tags.append((word, state_max))
    return list(zip(words, state)), unknown_word_tags

### Solve the problem of unknown words

#### Method 1: A transition probability based approach

This flavour considers the maximum of transition probability for tagging the unknown words.
<p>Steps:
* Both Emission and Transition probability is calculated for a word
* If a word has the max probability as 0, this means that the Emission probability is 0 because none of the Transition probabilities are 0 as per the tags-matrix. Effectively, the word is unknown.
* In this case, tag having the maximum Transition probability is considered as the POS tag for the unknown word. 

In [51]:
def Viterbi_TransitionBased(words, T = tags):
    state = []
    unknown_word_tags = []
    #T = sorted(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        trans_probs = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities            
            word_given_tagResults = word_given_tag(words[key], tag)
            emission_p = word_given_tagResults[0]/word_given_tagResults[1]
            state_probability = emission_p * transition_p
            #Cache the transition probabilities also
            trans_probs.append(transition_p)
            p.append(state_probability)
            
        pmax = max(p)
        tmax = max(trans_probs)
        if pmax > 0:
        # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
        else:
            state_max = T[trans_probs.index(tmax)] 
            unknown_word_tags.append((word,state_max))
        state.append(state_max)
    return list(zip(words, state)), unknown_word_tags

#### Method 2: Another transition probability based approach

The Transition probability considers the probability of a **tag given a previous tag**. This is another flavour which considers the probability of a **tag given a following tag** also.
Maximum of transition probabilities and reverse transition probabilities is considered here for tagging the unknown words.
<p>Steps:
* Both Emission and Transition probability is calculated for a word
* If a word has the max probability as 0, this means that the Emission probability is 0 because none of the Transition probabilities are 0 as per the tags-matrix. Effectively, the word is unknown.
* After this the words are scanned again(disadvantage, but this method depends on knowing the next tag) and the probabilities are calculated for a tag given a following tag. Maximum of these is taken as the tag.

In [52]:
# compute tag given tag: tag2(t2) given tag1 (t1), where t2 is followed by t1 i.e. Reverse Transition Probability
def t2_given_t1Follows(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index+1]==t1 and tags[index] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [53]:
rev_tags_matrix = np.zeros((no_of_unique_tags, no_of_unique_tags), dtype='float32')
for i, t1 in enumerate(tags):
    for j, t2 in enumerate(tags): 
        t2_given_t1FollowsResult = t2_given_t1Follows(t2, t1)
        rev_tags_matrix[i, j] = t2_given_t1FollowsResult[0]/t2_given_t1FollowsResult[1]

In [54]:
# convert the matrix to a df for better readability
rev_tags_df = pd.DataFrame(rev_tags_matrix, columns = tags, index=tags)
rev_tags_df

,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
.,0.092866,0.035669,0.033598,0.036840,0.006665,0.012790,0.593226,0.035219,0.009097,0.011439,0.040173,0.092416
ADJ,0.080833,0.067388,0.163633,0.064437,0.041974,0.278570,0.054435,0.018364,0.031317,0.043614,0.138383,0.017052
ADP,0.107032,0.050902,0.016967,0.038203,0.012165,0.008324,0.517234,0.012912,0.006403,0.006723,0.126347,0.096788
ADV,0.192308,0.008952,0.041114,0.078581,0.039788,0.035809,0.153846,0.003316,0.029509,0.010279,0.351459,0.055040
CONJ,0.302142,0.047020,0.003724,0.009311,0.000466,0.001862,0.542365,0.021881,0.006518,0.003259,0.031657,0.029330
DET,0.233357,0.003739,0.366015,0.025084,0.030873,0.005548,0.044380,0.001447,0.003015,0.037627,0.207429,0.041486
NOUN,0.089809,0.155245,0.110431,0.003315,0.027398,0.192626,0.264145,0.043502,0.019164,0.027835,0.052392,0.014136
NUM,0.266074,0.035852,0.175111,0.029333,0.026667,0.055407,0.077333,0.183704,0.005926,0.052148,0.087407,0.005037
PRON,0.283489,0.001558,0.248442,0.018692,0.047508,0.010514,0.050234,0.001947,0.008178,0.021807,0.174065,0.133567
PRT,0.008841,0.021938,0.004584,0.014080,0.003602,0.000655,0.398166,0.030452,0.010478,0.001965,0.132286,0.372954


In [55]:
#Setting this to False for now to restrict some of the verbose information 
#about Transition Probability and Reverse Transition Probability
#Can be set to true later if required
print_tag_prob_comp = False

In [56]:
def Viterbi_TransitionBasedRev(words, T = tags):
    state = []
    unknown_word_tags = []
    max_probs = [] 
    max_trans_probs = []
    max_trans_probs_tuple = []
    trans_tags = []
    trans_prob_comparison = []
    #T = sorted(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        trans_probs = []
        trans_prob_tuple = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities            
            word_given_tagResults = word_given_tag(words[key], tag)
            emission_p = word_given_tagResults[0]/word_given_tagResults[1]
            state_probability = emission_p * transition_p
            trans_probs.append(transition_p)
            if key == 0:
                trans_prob_tuple.append('{0} followed by {1} with prob = {2}'.format('.', tag, transition_p))
            else:
                trans_prob_tuple.append('{0} followed by {1} with prob = {2}'.format(state[-1], tag, transition_p))
            p.append(state_probability)
            
        pmax = max(p)
        tmax = max(trans_probs)
        max_probs.append(pmax)
        max_trans_probs.append(tmax)
        trans_tags.append(T[trans_probs.index(tmax)])
        max_trans_probs_tuple.append(trans_prob_tuple[trans_probs.index(tmax)])
        state_max = T[p.index(pmax)]        
        state.append(state_max)
    
    for i, maxprob in enumerate(max_probs):
        if maxprob == 0: 
            rev_tag_probs = []
            rev_trans_prob_tuple = []
            for tag in T:
                transition_p = rev_tags_df.loc[state[i+1], tag]
                rev_tag_probs.append(transition_p)
                rev_trans_prob_tuple.append('{0} followed by {1} with prob = {2}'.format(tag, state[i+1], transition_p))
            inv_tmax = max(rev_tag_probs) 
            if inv_tmax > max_trans_probs[i]:                
                state[i] = T[rev_tag_probs.index(inv_tmax)]                
            else:  
                state[i] = trans_tags[i]
            trans_prob_comparison.append((words[i], max_trans_probs_tuple[i], 
                            rev_trans_prob_tuple[rev_tag_probs.index(inv_tmax)], 'Applying {0}'.format(state[i])))    
            unknown_word_tags.append((words[i],state[i]))
    if print_tag_prob_comp: 
        print("Tag Applied with Probability comparisons = ", trans_prob_comparison)
        print("")
    return list(zip(words, state)), unknown_word_tags

#### Method 3: A rule based approach

Another set of modifications to the Vanilla Viterbi algorithm where it applies some rules for tagging the unknown words
The rules are applied in the following order:
* Rule 1: All words having one or more digits are tagged as NUM
* Rule 2: All words starting with one or more star are tagged as X 
* Rule 3: All words starting with numbers and ending with st,nd,rd,th are tagged as ADJ
* Rule 4: All words starting with caps in the middle of a sentence are tagged as NOUN
<p>The next 4 rules are taken from https://dictionary.cambridge.org/grammar/british-grammar/word-formation/suffixes
* Rule 5: All words ending with ly|wards|wise are tagged as ADV
* Rule 6: All words ending with ate|en|ify|ise|ize|ed|ing are tagged as VERB
* Rule 7: All words ending with able|ible|al|en|ese|ful|i|ic|ish|ive|ian|less|ous are tagged as ADJ
* Rule 8: All words ending with age|al|ance|ence|dom|ee|er|or|hood|ism|ist|ity|ty|ment|ness|ry|ship|sion|tion|xion are tagged as NOUN
* Rule 8: If a word does not belong to any of the above rules, then it probably is some Name and hence default tagging done is a NOUN

In [57]:
def Viterbi_RuleBased(words, T = tags):
    state = []
    unknown_word_tags = []
    #T = sorted(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc[tag, '.']
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            word_given_tagResults = word_given_tag(words[key], tag)
            emission_p = word_given_tagResults[0]/word_given_tagResults[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p) 
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        if pmax == 0:            
            if re.match('^(\d)[0-9.]+(\d)$', word):
                state_max = 'NUM' 
            elif re.match('^\*+.*', word):
                state_max = 'X'
            elif re.match('^(\d)[0-9]+(st|nd|rd|th)$',word):
                state_max = 'ADJ'
            elif word[0].isupper() and words[key-1] != '.':
                state_max = 'NOUN'    
            elif re.match('.+(ly|wards|wise)$', word):
                state_max = 'ADV'
            elif re.match('.+(ate|en|ify|ise|ize|ed|ing)$', word):
                state_max = 'VERB' 
            elif re.match('.+(able|ible|al|en|ese|ful|i|ic|ish|ive|ian|less|ous)$', word):
                state_max = 'ADJ'
            elif re.match('.+(age|al|ance|ence|dom|ee|er|or|hood|ism|ist|ity|ty|ment|ness|ry|ship|sion|tion|xion)$', word):
                state_max = 'NOUN'                           
            else: 
                state_max = 'NOUN'                
            unknown_word_tags.append((word, state_max))          
        state.append(state_max)
    return list(zip(words, state)), unknown_word_tags

#### Evaluating tagging accuracy

Function defined to set up the test data

In [58]:
def setup_test_data(test_set, no_of_sent):
    random.seed(1234)
    # choose random 5 sents
    rndom = [random.randint(1,len(test_set)) for x in range(no_of_sent)]
    # list of sents
    test_run = [test_set[i] for i in rndom]
    # list of tagged words
    test_run_base = [tup for sent in test_run for tup in sent]
    # list of untagged words
    test_tagged_words = [tup[0] for sent in test_run for tup in sent]
    return test_run, test_tagged_words, test_run_base  

Function defined to:
* Perform tagging by calling the passed algorithm - A function reference is passed as parameter here
* Record the time taken in seconds
* Conditionally calculate the accuracy based on whether a test_run_base with tagged words is available

In [59]:
def tag_input_words(fn, test_tagged_words, test_run_base = None):
    start = time.time()
    tagged_seq, unknown_word_tags = fn(test_tagged_words)
    end = time.time()
    difference = end-start

    # accuracy
    accuracy = None
    if test_run_base !=  None:
        check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
        accuracy = len(check)/len(tagged_seq)
    
    return accuracy, difference, unknown_word_tags, tagged_seq

In [60]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
test_run, test_tagged_words, test_run_base = setup_test_data(test_set, 5)
test_tagged_words

['``',
 'The',
 'disturbing',
 'thing',
 'about',
 'this',
 'abortion',
 'issue',
 'is',
 'that',
 'the',
 'debate',
 'has',
 'become',
 'polarized',
 '*-1',
 ',',
 'so',
 'that',
 'no',
 'mechanism',
 '*ICH*-2',
 'exists',
 "''",
 'for',
 '*',
 'finding',
 'a',
 'middle',
 'ground',
 '.',
 'The',
 'judge',
 'declined',
 '*-1',
 'to',
 'discuss',
 'his',
 'salary',
 'in',
 'detail',
 ',',
 'but',
 'said',
 ':',
 '``',
 'I',
 "'m",
 'going',
 '*-2',
 'to',
 'be',
 'a',
 'high-priced',
 'lawyer',
 '.',
 "''",
 'The',
 'Japanese',
 'retort',
 'that',
 'the',
 'first',
 'round',
 'was',
 'too',
 'early',
 '*',
 'to',
 'make',
 'concessions',
 '.',
 '``',
 'This',
 'conforms',
 'to',
 'the',
 '`',
 'soft',
 'landing',
 "'",
 'scenario',
 ',',
 "''",
 'said',
 '*T*-1',
 'Elliott',
 'Platt',
 ',',
 'an',
 'economist',
 'at',
 'Donaldson',
 ',',
 'Lufkin',
 '&',
 'Jenrette',
 'Securities',
 'Corp',
 '.',
 'In',
 'its',
 'construction',
 'spending',
 'report',
 ',',
 'the',
 'Commerce',
 'Depar

In [61]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_Vanilla, test_tagged_words, test_run_base)
print('Viterbi Vanilla Results=========================>')
print("")
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Viterbi Vanilla Results=========================>

Accuracy =  0.89

Time Taken in seconds =  13.7

Unknown word tags =  [('polarized', '.'), ('exists', '.'), ('detail', '.'), ('retort', '.'), ('concessions', '.'), ('conforms', '.'), ('Elliott', '.'), ('191.9', '.')]

Final tagging result =  [('``', '.'), ('The', 'DET'), ('disturbing', 'ADJ'), ('thing', 'NOUN'), ('about', 'ADP'), ('this', 'DET'), ('abortion', 'NOUN'), ('issue', 'NOUN'), ('is', 'VERB'), ('that', 'ADP'), ('the', 'DET'), ('debate', 'NOUN'), ('has', 'VERB'), ('become', 'VERB'), ('polarized', '.'), ('*-1', 'X'), (',', '.'), ('so', 'ADV'), ('that', 'ADP'), ('no', 'DET'), ('mechanism', 'NOUN'), ('*ICH*-2', 'X'), ('exists', '.'), ("''", '.'), ('for', 'ADP'), ('*', 'X'), ('finding', 'VERB'), ('a', 'DET'), ('middle', 'NOUN'), ('ground', 'NOUN'), ('.', '.'), ('The', 'DET'), ('judge', 'NOUN'), ('declined', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('discuss', 'VERB'), ('his', 'PRON'), ('salary', 'NOUN'), ('in', 'ADP'), ('detail', '.')

In [62]:
gc.collect()

242

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

#### Viterbi Plain with Transition Based Results

In [63]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_TransitionBased, test_tagged_words, test_run_base)
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Accuracy =  0.91

Time Taken in seconds =  13.12

Unknown word tags =  [('polarized', 'X'), ('exists', 'VERB'), ('detail', 'DET'), ('retort', 'NOUN'), ('concessions', 'X'), ('conforms', 'NOUN'), ('Elliott', 'VERB'), ('191.9', 'NOUN')]

Final tagging result =  [('``', '.'), ('The', 'DET'), ('disturbing', 'ADJ'), ('thing', 'NOUN'), ('about', 'ADP'), ('this', 'DET'), ('abortion', 'NOUN'), ('issue', 'NOUN'), ('is', 'VERB'), ('that', 'ADP'), ('the', 'DET'), ('debate', 'NOUN'), ('has', 'VERB'), ('become', 'VERB'), ('polarized', 'X'), ('*-1', 'X'), (',', '.'), ('so', 'ADV'), ('that', 'ADP'), ('no', 'DET'), ('mechanism', 'NOUN'), ('*ICH*-2', 'X'), ('exists', 'VERB'), ("''", '.'), ('for', 'ADP'), ('*', 'X'), ('finding', 'VERB'), ('a', 'DET'), ('middle', 'NOUN'), ('ground', 'NOUN'), ('.', '.'), ('The', 'DET'), ('judge', 'NOUN'), ('declined', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('discuss', 'VERB'), ('his', 'PRON'), ('salary', 'NOUN'), ('in', 'ADP'), ('detail', 'DET'), (',', '.'), ('but', 'CONJ'

#### Viterbi Plain with Reverse Transition Based Results

In [64]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_TransitionBasedRev, test_tagged_words, test_run_base)
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Accuracy =  0.92

Time Taken in seconds =  13.02

Unknown word tags =  [('polarized', 'VERB'), ('exists', 'NOUN'), ('detail', 'NOUN'), ('retort', 'NOUN'), ('concessions', 'NOUN'), ('conforms', 'NOUN'), ('Elliott', 'NOUN'), ('191.9', '.')]

Final tagging result =  [('``', '.'), ('The', 'DET'), ('disturbing', 'ADJ'), ('thing', 'NOUN'), ('about', 'ADP'), ('this', 'DET'), ('abortion', 'NOUN'), ('issue', 'NOUN'), ('is', 'VERB'), ('that', 'ADP'), ('the', 'DET'), ('debate', 'NOUN'), ('has', 'VERB'), ('become', 'VERB'), ('polarized', 'VERB'), ('*-1', 'X'), (',', '.'), ('so', 'ADV'), ('that', 'ADP'), ('no', 'DET'), ('mechanism', 'NOUN'), ('*ICH*-2', 'X'), ('exists', 'NOUN'), ("''", '.'), ('for', 'ADP'), ('*', 'X'), ('finding', 'VERB'), ('a', 'DET'), ('middle', 'NOUN'), ('ground', 'NOUN'), ('.', '.'), ('The', 'DET'), ('judge', 'NOUN'), ('declined', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('discuss', 'VERB'), ('his', 'PRON'), ('salary', 'NOUN'), ('in', 'ADP'), ('detail', 'NOUN'), (',', '.'), ('but'

#### Viterbi plain with Rule Based Results

In [65]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_RuleBased, test_tagged_words, test_run_base)
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Accuracy =  0.94

Time Taken in seconds =  13.06

Unknown word tags =  [('polarized', 'VERB'), ('exists', 'NOUN'), ('detail', 'NOUN'), ('retort', 'NOUN'), ('concessions', 'NOUN'), ('conforms', 'NOUN'), ('Elliott', 'NOUN'), ('191.9', 'NUM')]

Final tagging result =  [('``', '.'), ('The', 'DET'), ('disturbing', 'ADJ'), ('thing', 'NOUN'), ('about', 'ADP'), ('this', 'DET'), ('abortion', 'NOUN'), ('issue', 'NOUN'), ('is', 'VERB'), ('that', 'ADP'), ('the', 'DET'), ('debate', 'NOUN'), ('has', 'VERB'), ('become', 'VERB'), ('polarized', 'VERB'), ('*-1', 'X'), (',', '.'), ('so', 'ADV'), ('that', 'ADP'), ('no', 'DET'), ('mechanism', 'NOUN'), ('*ICH*-2', 'X'), ('exists', 'NOUN'), ("''", '.'), ('for', 'ADP'), ('*', 'X'), ('finding', 'VERB'), ('a', 'DET'), ('middle', 'NOUN'), ('ground', 'NOUN'), ('.', '.'), ('The', 'DET'), ('judge', 'NOUN'), ('declined', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('discuss', 'VERB'), ('his', 'PRON'), ('salary', 'NOUN'), ('in', 'ADP'), ('detail', 'NOUN'), (',', '.'), ('bu

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

#### Read file of sample sentences and setup for testing

In [66]:
with open('Test_sentences.txt', 'r') as f:
    sample_test_sentences = f.read().splitlines()
sample_test_sentences = list(filter(None, sample_test_sentences))
sample_test_sentences

['Android is a mobile operating system developed by Google.',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.',
 'Before entering politics, Donald Trump was a domineering businessman and a television personality.',
 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.',
 'Show me the cheapest round trips from Dallas to Atlanta',
 'I would like to see flights from Denver to Philadelphia.',
 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.',
 'NASA invited social media users to experienc

#### Handle puncutations and apostrophe s
Parse all the sentences into words along with special cases like full-stop, apostrophe s, comma as a different words.
Put a space before punctuations and 's so that they get split as different words

In [67]:
sample_sent_list_words = []
for sent in sample_test_sentences:
    sent = sent.replace('.', ' .')
    sent = sent.replace('\'s', ' \'s')
    sent = sent.replace(',', ' ,')
    sent_words = sent.split()
    sample_sent_list_words.append(sent_words)
    
sample_sent_list_words

[['Android',
  'is',
  'a',
  'mobile',
  'operating',
  'system',
  'developed',
  'by',
  'Google',
  '.'],
 ['Android',
  'has',
  'been',
  'the',
  'best-selling',
  'OS',
  'worldwide',
  'on',
  'smartphones',
  'since',
  '2011',
  'and',
  'on',
  'tablets',
  'since',
  '2013',
  '.'],
 ['Google',
  'and',
  'Twitter',
  'made',
  'a',
  'deal',
  'in',
  '2015',
  'that',
  'gave',
  'Google',
  'access',
  'to',
  'Twitter',
  "'s",
  'firehose',
  '.'],
 ['Twitter',
  'is',
  'an',
  'online',
  'news',
  'and',
  'social',
  'networking',
  'service',
  'on',
  'which',
  'users',
  'post',
  'and',
  'interact',
  'with',
  'messages',
  'known',
  'as',
  'tweets',
  '.'],
 ['Before',
  'entering',
  'politics',
  ',',
  'Donald',
  'Trump',
  'was',
  'a',
  'domineering',
  'businessman',
  'and',
  'a',
  'television',
  'personality',
  '.'],
 ['The',
  '2018',
  'FIFA',
  'World',
  'Cup',
  'is',
  'the',
  '21st',
  'FIFA',
  'World',
  'Cup',
  ',',
  'an',
  'i

In [68]:
def sample_sent_test(test_sentence):
    test_words = [word for word in test_sentence]
    print("Test sentence = ", test_sentence)
    print("")
    accuracy, difference, vanilla_unknown_word_tags, vanilla_tagged_seq = tag_input_words(Viterbi_Vanilla, test_words)
    print("Vanilla Unknown word tags = ", vanilla_unknown_word_tags)
    print("")
    print("Vanilla Final Tagging Result = ", vanilla_tagged_seq)
    print("")
    accuracy, difference, trans_unknown_word_tags, trans_tagged_seq = tag_input_words(Viterbi_TransitionBased, test_words)
    print("Method 1:Transition Based Unknown word tags = ", trans_unknown_word_tags)
    print("")
    print("Method 1:Transition Based Final Tagging Result = ", trans_tagged_seq)
    print("")
    accuracy, difference, revtrans_unknown_word_tags, revtrans_tagged_seq = tag_input_words(Viterbi_TransitionBasedRev, test_words)
    print("Method 2:Rev Transition Based Unknown word tags = ", revtrans_unknown_word_tags)
    print("")
    print("Method 2:Rev Transition Based Final Tagging Result = ", revtrans_tagged_seq)
    print("")
    accuracy, difference, rule_unknown_word_tags, rule_tagged_seq = tag_input_words(Viterbi_RuleBased, test_words)
    print("Method 3:Rule Based Unknown word tags = ", rule_unknown_word_tags)
    print("")
    print("Method 3:Rule Based Final Tagging Result = ", revtrans_tagged_seq)

In [69]:
sample_sent_test(sample_sent_list_words[1])
print('======================================================================================')
sample_sent_test(sample_sent_list_words[5])
print('======================================================================================')
sample_sent_test(sample_sent_list_words[10])

Test sentence =  ['Android', 'has', 'been', 'the', 'best-selling', 'OS', 'worldwide', 'on', 'smartphones', 'since', '2011', 'and', 'on', 'tablets', 'since', '2013', '.']

Vanilla Unknown word tags =  [('Android', '.'), ('OS', '.'), ('worldwide', '.'), ('smartphones', '.'), ('2011', '.'), ('tablets', '.'), ('2013', '.')]

Vanilla Final Tagging Result =  [('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', '.'), ('since', 'ADP'), ('2013', '.'), ('.', '.')]

Method 1:Transition Based Unknown word tags =  [('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', 'DET'), ('2011', 'DET'), ('tablets', 'DET'), ('2013', 'DET')]

Method 1:Transition Based Final Tagging Result =  [('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), 

### Sample data tag set testing with accuracy

The sentences from the "Sample test sentences" have been taken and tagged, so that accuracy can be calculated along with the pointing out the correctly tagged words because of the modified Viterbi algorithms

In [70]:
sample_tagged_words = [
 [('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')],
 [('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')],
 [('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ('\'s', 'PRT'), ('firehose', 'NOUN'), ('.', '.')],
 [('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'ADJ'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOUN'), ('post', 'VERB'), ('and', 'CONJ'), ('interact', 'VERB'), ('with', 'ADP'), ('messages', 'NOUN'), ('known', 'VERB'), ('as', 'ADP'), ('tweets', 'NOUN'), ('.', '.')],
 [('Before', 'ADP'), ('entering', 'VERB'), ('politics', 'NOUN'), (',', '.'), ('Donald', 'NOUN'), ('Trump', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('domineering', 'ADJ'), ('businessman', 'NOUN'), ('and', 'CONJ'), ('a', 'DET'), ('television', 'NOUN'), ('personality', 'NOUN'), ('.', '.')],
 [('The', 'DET'), ('2018', 'NUM'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'ADJ'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')],
 [('This', 'DET'), ('is', 'VERB'), ('the', 'DET'), ('first', 'ADJ'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('to', 'PRT'), ('be', 'VERB'), ('held', 'VERB'), ('in', 'ADP'), ('Eastern', 'NOUN'), ('Europe', 'NOUN'), ('and', 'CONJ'), ('the', 'DET'), ('11th', 'ADJ'), ('time', 'NOUN'), ('that', 'ADP'), ('it', 'PRON'), ('has', 'VERB'), ('been', 'VERB'), ('held', 'VERB'), ('in', 'ADP'), ('Europe', 'NOUN'), ('.', '.')],
 [('Show', 'VERB'), ('me', 'PRON'), ('the', 'DET'), ('cheapest', 'ADJ'), ('round', 'NOUN'), ('trips', 'NOUN'), ('from', 'ADP'), ('Dallas', 'NOUN'), ('to', 'PRT'), ('Atlanta', 'NOUN')],
 [('I', 'PRON'), ('would', 'VERB'), ('like', 'ADP'), ('to', 'PRT'), ('see', 'VERB'), ('flights', 'NOUN'), ('from', 'ADP'), ('Denver', 'NOUN'), ('to', 'PRT'), ('Philadelphia', 'NOUN'), ('.', '.')],
 [('Show', 'VERB'), ('me', 'PRON'), ('the', 'DET'), ('price', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('flights', 'NOUN'), ('leaving', 'VERB'), ('Atlanta', 'NOUN'), ('at', 'ADP'), ('about', 'ADP'), ('3', 'NUM'), ('in', 'ADP'), ('the', 'DET'), ('afternoon', 'NOUN'), ('and', 'CONJ'), ('arriving', 'VERB'), ('in', 'ADP'), ('San', 'NOUN'), ('Francisco', 'NOUN'), ('.', '.')],
 [('NASA', 'NOUN'), ('invited', 'VERB'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'VERB'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN'), ('.', '.')]]

In [71]:
## test_run_base = sample_tagged_words[0:3]
test_run = []
test_run.append(sample_tagged_words[1])
test_run.append(sample_tagged_words[5])
test_run.append(sample_tagged_words[10])
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
# list of untagged words
test_tagged_words = [tup[0] for tup in test_run_base]

In [72]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_Vanilla, test_tagged_words, test_run_base)
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Accuracy =  0.62

Time Taken in seconds =  4.92

Unknown word tags =  [('Android', '.'), ('OS', '.'), ('worldwide', '.'), ('smartphones', '.'), ('2011', '.'), ('tablets', '.'), ('2013', '.'), ('2018', '.'), ('FIFA', '.'), ('Cup', '.'), ('21st', '.'), ('FIFA', '.'), ('Cup', '.'), ('tournament', '.'), ('contested', '.'), ('NASA', '.'), ('invited', '.'), ('ICESAT-2', '.'), ('Satellite', '.')]

Final tagging result =  [('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', '.'), ('since', 'ADP'), ('2013', '.'), ('.', '.'), ('The', 'DET'), ('2018', '.'), ('FIFA', '.'), ('World', 'NOUN'), ('Cup', '.'), ('is', 'VERB'), ('the', 'DET'), ('21st', '.'), ('FIFA', '.'), ('World', 'NOUN'), ('Cup', '.'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', '.'), ('contested', '.'),

In [73]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_TransitionBased, test_tagged_words, test_run_base)
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Accuracy =  0.81

Time Taken in seconds =  4.79

Unknown word tags =  [('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', 'DET'), ('2011', 'DET'), ('tablets', 'DET'), ('2013', 'DET'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'NOUN'), ('NASA', 'NOUN'), ('invited', 'NOUN'), ('ICESAT-2', 'DET'), ('Satellite', 'NOUN')]

Final tagging result =  [('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'DET'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.'), ('The', 'DET'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), (

In [74]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_TransitionBasedRev, test_tagged_words, test_run_base)
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Accuracy =  0.88

Time Taken in seconds =  4.87

Unknown word tags =  [('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', 'NOUN'), ('2011', 'NOUN'), ('tablets', 'NOUN'), ('2013', 'NOUN'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('NASA', 'NOUN'), ('invited', 'DET'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN')]

Final tagging result =  [('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('The', 'DET'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', '

In [75]:
accuracy, difference, unknown_word_tags, tagged_seq = tag_input_words(Viterbi_RuleBased, test_tagged_words, test_run_base)
print("Accuracy = ", round(accuracy,2))
print("")
print("Time Taken in seconds = ", round(difference,2))
print("")
print("Unknown word tags = ", unknown_word_tags)
print("")
print("Final tagging result = ", tagged_seq)

Accuracy =  0.98

Time Taken in seconds =  4.71

Unknown word tags =  [('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', 'NOUN'), ('2011', 'NUM'), ('tablets', 'NOUN'), ('2013', 'NUM'), ('2018', 'NUM'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('21st', 'ADJ'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('NASA', 'NOUN'), ('invited', 'VERB'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN')]

Final tagging result =  [('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('The', 'DET'), ('2018', 'NUM'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'ADJ'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), 

In [76]:
gc.collect()

0

### Conclusions

#### Accuracies for chosen test-set
<table align=left width=100%>
<thead>
<tr>
<th>Algorithm</th><th>Accuracy</th></tr>
</thead>
<tr><td>Vanilla Viterbi</td><td>89%</td></tr>
<tr><td>Vanilla Viterbi with max of Transition Based probability tagging for unknown words</td><td>91%</td></tr>
<tr><td>Vanilla Viterbi with max of Transition and Reverse Transition Based probability tagging for unknown words</td><td>92%</td></tr>
<tr><td>Vanilla Viterbi with Rule based tagging for unknown words</td><td>94%</td></tr>
</table>
<hr/>
<p>&nbsp;</p>
<p>
<h4>Example 3 cases of correction of POS tags for unknown words</h4>
<table align=left width=100%>
<thead>
<tr>
<th>Algorithm</th><th>Unknown word tags</th><th>Final tagging result</th></tr>
</thead>
<tr><td colspan=3><b>Test Sentence : Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.</b></td>
<tr><td>Vanilla Viterbi</td>
<td>[('Android', '.'), ('OS', '.'), ('worldwide', '.'), ('smartphones', '.'), ('2011', '.'), ('tablets', '.'), ('2013', '.')]</td>
<td>[('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', '.'), ('since', 'ADP'), ('2013', '.'), ('.', '.')]</td>
</tr>
<tr><td>Vanilla Viterbi with max of Transition Based probability tagging for unknown words</td>
<td>[('Android', '<font color=green><b>NOUN</b></font>'), ('OS', '<font color=green><b>NOUN</b></font>'), ('worldwide', '<font color=green><b>NOUN</b></font>'), ('smartphones', '<font color=green><b>DET</b></font>'), ('2011', '<font color=green><b>DET</b></font>'), ('tablets', '<font color=green><b>DET</b></font>'), ('2013', '<font color=green><b>DET</b></font>')]</td>
<td>[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'DET'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.')]</td></tr>
<tr><td>Vanilla Viterbi with max of Transition and Reverse Transition Based probability tagging for unknown words</td><td>[('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', '<font color=green><b>NOUN</b></font>'), ('2011', '<font color=green><b>NOUN</b></font>'), ('tablets', '<font color=green><b>NOUN</b></font>'), ('2013', '<font color=green><b>NOUN</b></font>')]</td>
<td>[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.')]</td></tr>
<tr><td>Vanilla Viterbi with Rule based tagging for unknown words</td>
<td>[('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', 'NOUN'), ('2011', '<font color=green><b>NUM</b></font>'), ('tablets', 'NOUN'), ('2013', '<font color=green><b>NUM</b></font>')]</td>
<td>[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.')]</td></tr>

<tr><td colspan=3><b>Test sentence : The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.</b></td><tr>
<tr><td>Vanilla Viterbi</td><td>[('2018', '.'), ('FIFA', '.'), ('Cup', '.'), ('21st', '.'), ('FIFA', '.'), ('Cup', '.'), ('tournament', '.'), ('contested', '.')]</td>
<td>[('The', 'DET'), ('2018', '.'), ('FIFA', '.'), ('World', 'NOUN'), ('Cup', '.'), ('is', 'VERB'), ('the', 'DET'), ('21st', '.'), ('FIFA', '.'), ('World', 'NOUN'), ('Cup', '.'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', '.'), ('contested', '.'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]</td><tr>
<tr><td>Vanilla Viterbi with max of Transition Based probability tagging for unknown words</td>
<td>[('2018', '<font color=green><b>NOUN</b></font>'), ('FIFA', '<font color=green><b>NOUN</b></font>'), ('Cup', '<font color=green><b>NOUN</b></font>), ('21st', '<font color=green><b>NOUN</b></font>'), ('FIFA', '<font color=green><b>NOUN</b></font>'), ('Cup', '<font color=green><b>NOUN</b></font>'), ('tournament', '<font color=green><b>NOUN</b></font>'), ('contested', '<font color=green><b>NOUN</b></font>')]</td>
<td>[('The', 'DET'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'NOUN'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]</td><tr>
<tr><td>Vanilla Viterbi with max of Transition and Reverse Transition Based probability tagging for unknown words</td><td>[('2018', 'NOUN'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('tournament', 'NOUN'), ('contested', '<font color=green><b>VERB</b></font>')]</td>
<td>[('The', 'DET'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]</td><tr>
<tr><td>Vanilla Viterbi with Rule based tagging for unknown words</td>
<td>[('2018', '<font color=green><b>NUM</b></font>'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('21st', '<font color=green><b>ADJ</b></font>'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB')]</td>
<td>[('The', 'DET'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]</td><tr>

<tr><td colspan=3><b>Test sentence : NASA invited social media users to experience the launch of ICESAT-2 Satellite.</b></td><tr>
<tr><td>Vanilla Viterbi</td>
<td>[('NASA', '.'), ('invited', '.'), ('ICESAT-2', '.'), ('Satellite', '.')]</td>
<td>[('NASA', '.'), ('invited', '.'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', '.'), ('Satellite', '.'), ('.', '.')]</td><tr>
<tr><td>Vanilla Viterbi with max of Transition Based probability tagging for unknown words</td>
<td>[('NASA', '<font color=green><b>NOUN</b></font>'), ('invited', '<font color=green><b>NOUN</b></font>'), ('ICESAT-2', '<font color=green><b>DET</b></font>'), ('Satellite', '<font color=green><b>NOUN</b></font>')]</td>
<td>[('NASA', 'NOUN'), ('invited', 'NOUN'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'DET'), ('Satellite', 'NOUN'), ('.', '.')]</td><tr>
<tr><td>Vanilla Viterbi with max of Transition and Reverse Transition Based probability tagging for unknown words</td><td>[('NASA', 'NOUN'), ('invited', '<font color=green><b>DET</b></font>'), ('ICESAT-2', '<font color=green><b>NOUN</b></font>'), ('Satellite', 'NOUN')]</td>
<td>[('NASA', 'NOUN'), ('invited', 'DET'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN'), ('.', '.')]</td><tr>
<tr><td>Vanilla Viterbi with Rule based tagging for unknown words</td><td>[('NASA', 'NOUN'), ('invited', '<font color=green><b>VERB</b></font>'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN')]</td>
<td>[('NASA', 'NOUN'), ('invited', 'DET'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN'), ('.', '.')]</td><tr>
</table>
<hr/>
<p>&nbsp;</p>
<h4>Accuracies for the same Example 3 cases of correction of POS tags for unknown words</h4>
<table align=left width=100%>
<thead>
<tr>
<th>Algorithm</th><th>Accuracy</th></tr>
</thead>
<tr><td>Vanilla Viterbi</td><td>62%</td></tr>
<tr><td>Vanilla Viterbi with max of Transition Based probability tagging for unknown words</td><td>81%</td></tr>
<tr><td>Vanilla Viterbi with max of Transition and Reverse Transition Based probability tagging for unknown words</td><td>88%</td></tr>
<tr><td>Vanilla Viterbi with Rule based tagging for unknown words</td><td>98%</td></tr>
</table>